In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import Tensor, nn

##  Linear layer
PyTorch provides a linear layer class, but the action it performs is relatively simple. Below provide your own implementation of a linear-layer class:

In [8]:
import math

class MyLinear(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.weight = torch.randn(nout, nin)/math.sqrt(nin)
        self.bias = torch.zeros(nout)
        
    def forward(self, x):
        return x@self.weight.T+self.bias

In [9]:
lin = MyLinear(2,5)
x = torch.randn(10,2)
y = lin(x)

In [12]:
assert y.shape == torch.Size([10,5])

## Full DNN

Below, implement an `nn.Module` class that can provide a DNN that takes in 3 features and returns a single, sigmoided output, after passing through several layers of linear and activation layers.

In [4]:
class DNN(nn.Module):
    def __init__(self, sizes=[3,10,10,1]):
        super().__init__()
        self.layers = nn.Sequential(*[self.get_layer(sizes[i], sizes[i+1]) for i in range(len(sizes)-2)])
        self.out_layer = self.get_layer(sizes[-2], sizes[-1], out_layer=True)

    @staticmethod
    def get_layer(nin, nout, out_layer=False):
        lin = nn.Linear(nin, nout)
        if out_layer:
            nn.init.xavier_normal_(lin.weight)
            act = nn.Sigmoid()
        else:
            nn.init.kaiming_normal_(lin.weight, nonlinearity='relu')
            act = nn.ReLU()
        nn.init.zeros_(lin.bias)
        
        return nn.Sequential(lin, act)
    
    def forward(self, x):
        x = self.layers(x)
        return self.out_layer(x)

In [5]:
dnn = DNN()
x = torch.randn(100,3)

In [8]:
preds = dnn(x)
assert preds.shape == torch.Size([100,1])
assert (preds >= 0).all()
assert (preds <= 1).all()